In [30]:
import models.vae_mnist as vae_mnist
from torchvision import transforms, datasets
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

latent_dim = 20

conv_vae = vae_mnist.ConvVAE(latent_dim).to(device)
conv_vae.load_state_dict(torch.load('conv_vae_mnist.pth', map_location='cpu'))

<All keys matched successfully>

In [61]:
activations_encoder = {
    'layer1' : None, 
    'layer2' : None, 
    'layer3' : None, 
    'layer4' : None,
}

activations_decoder = {
    'layer1' : None, 
    'layer2' : None, 
    'layer3' : None, 
    'layer4' : None,
}
# activations_encoder = {}
def get_activation_foo(name, activations):
    def hookFoo(model, input, output):
        if activations[name] == None: 
            activations[name] = output.detach()
        else:
            activations[name] = torch.cat((activations[name], output.detach()), 0)
    return hookFoo

def get_activation_foo_input(name, activations):
    def hookFoo(model, input, output):
        if activations[name] == None: 
            activations[name] = input[0].detach()
        else:
            activations[name] = torch.cat((activations[name], input[0].detach()), 0)
    return hookFoo

conv_vae.encoder[0].register_forward_hook(get_activation_foo('layer1', activations_encoder))
conv_vae.encoder[2].register_forward_hook(get_activation_foo('layer2', activations_encoder))
conv_vae.encoder[4].register_forward_hook(get_activation_foo('layer3', activations_encoder))
conv_vae.encoder[9].register_forward_hook(get_activation_foo('layer4', activations_encoder))

conv_vae.decoder[0].register_forward_hook(get_activation_foo_input('layer1', activations_decoder))
conv_vae.decoder[5].register_forward_hook(get_activation_foo_input('layer2', activations_decoder))
conv_vae.decoder[7].register_forward_hook(get_activation_foo_input('layer3', activations_decoder))
conv_vae.decoder[9].register_forward_hook(get_activation_foo_input('layer4', activations_decoder))

In [51]:
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset, Subset
from smallnet import LinearNet, Net, split_dataset

In [63]:
batch_size = 32

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: 2*x - 1)  # Normalize MNIST data
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_indices, test_indices = split_dataset(train_dataset, num_train_per_class=32, num_test_per_class=64)

train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(test_dataset, test_indices)

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=batch_size)
     

In [64]:
def transform_to_features(activations, batch_size):
    with torch.no_grad():
        feats = torch.Tensor([])
        for val in activations.values():
            print(val)
            cur_feat = val[-batch_size:]
            if len(val.shape) == 4:
                cur_feat = val[-batch_size:].mean(dim=[2,3])
            print(feats.shape)
            print(cur_feat.shape)
            feats = torch.cat((feats, cur_feat), dim=1)
            print(feats.shape)
            print('********************')
    return feats

In [43]:
a=np.array([[[0,1,2],[0,3,4]], [[0,1,2],[0,3,4]], [[0,1,2],[0,3,4]]])
a[-1:]

array([[[0, 1, 2],
        [0, 3, 4]]])

In [65]:
features_umap = []
conv_vae.to(device)
for images, labels in train_dataset:
    print('NEW IMG')
    images, labels = images.to(device), torch.tensor(labels).to(device)
    with torch.no_grad():
        print(images.shape)
        conv_vae.to(device)
        conv_vae(images)
    print('LABELS: ', labels)
    features_enc = transform_to_features(activations_encoder, labels)
    features_umap.append(features_enc.cpu().detach().numpy().flatten())

NEW IMG
torch.Size([1, 28, 28])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x49 and 6272x1024)